In [1]:
'''
GA Data Science Q2 2016

Code walk-through 18: Databases and SQL
'''

import os

import numpy as np
import pandas as pd

import sqlite3

In [4]:
CYCLE_HIRES_URL = 'https://files.datapress.com/london/dataset/number-bicycle-hires/2016-05-17T09:24:57/tfl-daily-cycle-hires.xls'

# Load TfL Cycle Hire dataset
hires = pd.read_excel(CYCLE_HIRES_URL, sheetname='Data')

# Convert 'Day' to 'datetime' and set as index
hires.Day = pd.to_datetime(hires.Day, unit='D')
hires.set_index('Day', inplace=True)

# Extract first column (daily hires) and convert to 'float'
hires = hires.iloc[:,0].astype('float').to_frame('Hires')

# Write DataFrame to disk
db_connection = sqlite3.connect('../../Data/tfl_cycle_hire.db')
hires.to_sql('hires', con=db_connection, if_exists='replace')

In [5]:
# Retrieve first 10 records, output will be a dataframe.
pd.io.sql.read_sql('SELECT * FROM hires LIMIT 10', con=db_connection)

,Day,Hires
0,2010-07-30 00:00:00,6897.0
1,2010-07-31 00:00:00,5564.0
2,2010-08-01 00:00:00,4303.0
3,2010-08-02 00:00:00,6642.0
4,2010-08-03 00:00:00,7966.0
5,2010-08-04 00:00:00,7893.0
6,2010-08-05 00:00:00,8724.0
7,2010-08-06 00:00:00,9797.0
8,2010-08-07 00:00:00,6631.0
9,2010-08-08 00:00:00,7864.0


In [6]:
# Retrieve all records from 2016
pd.io.sql.read_sql(
    '''SELECT Hires
       FROM hires
       WHERE STRFTIME(\'%Y\', Day) == \'2016\'''', con=db_connection)

,Hires
0,9922.0
1,7246.0
2,4894.0
3,20644.0
4,22934.0
5,23199.0
6,18225.0
7,20948.0
8,11674.0
9,14447.0


In [7]:
# Compute average number of cycle hires by month
pd.io.sql.read_sql(
    '''SELECT Day, AVG(Hires) AS average_hires
       FROM hires
       GROUP BY STRFTIME(\'%m\', Day)''', con=db_connection)

,Day,average_hires
0,2016-01-31 00:00:00,16811.559140
1,2016-02-29 00:00:00,18038.617647
2,2016-03-31 00:00:00,21519.935484
3,2016-04-30 00:00:00,24381.694444
4,2015-05-31 00:00:00,27029.909677
5,2015-06-30 00:00:00,29356.626667
6,2015-07-31 00:00:00,32177.337580
7,2015-08-31 00:00:00,27683.370968
8,2015-09-30 00:00:00,27247.894444
9,2015-10-31 00:00:00,24610.790323


In [8]:
# Aggregate records by year and month, then sort by total number of cycle hires
pd.io.sql.read_sql(
    '''SELECT Day, SUM(Hires) AS total_hires
       FROM hires
       GROUP BY STRFTIME(\'%Y%m\', Day)
       ORDER BY total_hires DESC''', con=db_connection)

,Day,total_hires
0,2014-07-31 00:00:00,1183182.0
1,2012-08-31 00:00:00,1163171.0
2,2015-07-31 00:00:00,1134816.0
3,2014-09-30 00:00:00,1058684.0
4,2014-08-31 00:00:00,1055206.0
5,2014-06-30 00:00:00,1052573.0
6,2015-08-31 00:00:00,1042932.0
7,2015-06-30 00:00:00,1039768.0
8,2012-09-30 00:00:00,1015145.0
9,2012-07-31 00:00:00,1014174.0
